# Get Background Cell Images
## Imports

In [ ]:
import cv2
import numpy as np
import os

from config import *
from CellProcessor import read_image, process_image, get_bboxes, get_label_yolo

## Make dir for images

In [ ]:
IMAGES_BACKGROUND_DIR_PATH = IMAGES_PATH + "../"  + "Background_10p/"

try:
    os.mkdir(IMAGES_BACKGROUND_DIR_PATH)
except OSError:
    print("Dir already exists: " + IMAGES_BACKGROUND_DIR_PATH)

## For every image get iamges with no died cells

In [ ]:
images = os.listdir(GREEN_PATH)
len_images = len(images)
curr_image_num = 1
num_images = 0

for image in images:
    print(str(curr_image_num) + "/" + str(len_images))
    curr_image_num+=1
    green_image_path = GREEN_PATH + image
    base_image_path = PHASE_PATH + image
    img, img_base = read_image(green_image_path, base_image_path)
    img_green = cv2.imread(green_image_path, cv2.IMREAD_UNCHANGED)

    img_base_h, img_base_w = img_base.shape[:2]

    CROP_WIDTH = 128
    CROP_HEIGHT = 128
    CROP_WIDTH_OVERLAP = 64
    CROP_HEIGHT_OVERLAP = 52
    CROP_WIDTH_SHIFT = CROP_WIDTH - CROP_WIDTH_OVERLAP
    CROP_HEIGHT_SHIFT = CROP_HEIGHT - CROP_HEIGHT_OVERLAP
    num_width_windows = (img_base_w - CROP_WIDTH)/CROP_WIDTH_SHIFT
    num_height_windows = (img_base_h - CROP_HEIGHT)/CROP_HEIGHT_SHIFT

    for width_window in range(int(num_width_windows)):
        for height_window in range(int(num_height_windows)):
            curr_topleft_x = width_window * CROP_WIDTH_SHIFT
            curr_topleft_y = height_window * CROP_HEIGHT_SHIFT
            img_crop = img[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_base_crop = img_base[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_green_crop = img_green[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]

            img_dilation = process_image(img_crop, 
                                contrast, 
                                brightness, 
                                threshold, 
                                i_erode, 
                                i_dialate, 
                                plot = False)          

            bboxes = get_bboxes(img_dilation)
            mean_img = (np.sum(img_dilation)/4177920) * 100.0
            if len(bboxes) == 0 and mean_img < 40.0 and num_images < 10000:
                num_images+=1          
                img_file = IMAGES_BACKGROUND_DIR_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                cv2.imwrite(img_file, img_base_crop)